In [1]:
import os
import json

#openai_api_key = 'zk-a8cd1872858119e19c7f937d62598898'#国内openai代理
#openai_api_base = 'https://flag.smarttrot.com/v1'#国内openai代理

openai_api_key = '7ae2c82d-9e9d-4efa-8888-36cdbc5a09a8'#chatGLM3接口
openai_api_base = 'https://u130964-b408-8d4b0e7e.beijinga.seetacloud.com/v1'#chatGLM3接口

os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['OPENAI_API_BASE'] = openai_api_base

In [2]:
# 外部函数
import requests
import openai

def google_search(question):
    params = {
        "engine": "google",
        "q": question,
        "api_key": "c2ade14f2a86418829d40ef07de1bef97ae335d7c84942d62821f51b4601bf0f"
    }

    # 通常，您需要找到正确的URL端点进行请求，以下为示例
    url = "https://serpapi.com/search"

    response = requests.get(url, params=params)

    if response.status_code == 200:
            search_results = response.json()

            # 提取有机搜索结果
            organic_results = search_results.get("organic_results", [])
            results_list = [{"title": result.get("title"), "link": result.get("link")} for result in organic_results]
            
            return json.dumps(results_list, ensure_ascii=False)
    else:
        return json.dumps({"Error": response.status_code})

os.environ['OPENWEATHER_API_KEY'] = 'f93a12279449fb568920e3b54fe0c6e7'
open_weather_key = os.getenv("OPENWEATHER_API_KEY")

def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"
    # Step 2.设置查询参数
    params = {
        "q": loc, 
        "appid": open_weather_key, # 输入API key
        "units": "metric", # 使用摄氏度而不是华氏度
        "lang":"zh_cn" # 输出语言为简体中文
    }
    # Step 3.发送GET请求
    response = requests.get(url, params=params)

    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

def extract_info(user_input):
    prompt1 = '''
    你是一个专门用于提取关键信息的AI。根据以下输入信息，详细提取以下几个方面的信息，并以列表的形式回答：
    1. 活动名称
    2. 活动地点
    3. 活动目的
    4. 发言人
    5. 重要嘉宾
    6. 背景信息（包括但不限于活动的日期、内容、组织单位等额外信息）

    如果有任何信息无法从输入中找到，请用XXX表示。
    请仔细检查有没有其他背景信息。

    输入信息：“{}”

    请确保你的答案仅基于输入信息，不添加任何外来内容，不要编造背景信息。
    '''.format(user_input)
    return llm_6b(prompt1)

def inquiry(info):
    prompt2 = '''
    你是一个帮助用户写政府公文的指导老师。
    信息条目：[活动名称、活动地点、活动目的、发言人、重要嘉宾、背景信息]
    在以下输入信息条目中，还可以补充哪些信息？这些信息将用于公文写作。

    注意：不是直接帮助补充信息！

    请遵守以下格式：

    **
    为了更好地完成您的需求，我们建议您提供补充信息。

    以下是关于可补充信息的建议：
    1.
    2.
    3.
    **

    输入信息：“{}”
    '''.format(info)
    return llm_6b(prompt2)

def opinion(info):
    prompt3 = '''
    根据以下写作意图和主题，提出一些观点，要有创新性和启发性。
    必须按以下格式回复：
    1.……
    2.……
    
    输入信息：{}
    '''.format(info)
    return llm_6b(prompt3)

def article(info):
    prompt4 = '''
    <article>
    【称谓】尊敬的各位领导，各位来宾，女士们，先生们，朋友们：
    【欢迎】大家好！非常荣幸受邀参加本次大会，和大家一起见证和探讨我市发展的成果和前景。我谨代表市委市政府，向出席本次大会的各位嘉宾和所有关心和支持的各位朋友，表示热烈的欢迎和诚挚的问候！
    【目的】今天，我们召开此次大会，旨在推动我市实现更高质量、更有效率、更加公平、更可持续、更为安全的发展。
    【站位引用】习近平总书记深刻指出，……
    【论点】习近平总书记的重要讲话对……作了全面深刻阐述，我们深刻领悟到开放是人类文明进步的重要动力，是世界繁荣发展的必由之路。
    【发展历程】近年来，我市始终坚持既定发展战略，推动发展建设不断强化，实现……，呈现……的态势。
    【现状】当前，我市面临着……的严峻形势。为此，我们要以……纾发展之困、以……汇合作之力、以……聚创新之势、以……谋共享之福，推动经济发展不断向前，让发展成果更多更公平惠及人民。
    【做法1】我们将推动各方共享……机遇，加快建设……，推动……优化升级，创新……发展机制，扩大……窗口，创建……先行区，建设……示范区，推动……发展，推进高质量……。
    【做法2】我们将推动各方共享……机遇，稳步扩大……等制度，实施好……，深化……综合示范区建设；实施……战略，加快建设……，发挥好……的平台作用。
    【做法3】我们将推动各方共享……机遇，全面深入参与……改革，推动……自由化便利化，促进……政策协调，共同培育……新动能，积极推进文明建设，扩大高标准……网络，坚定支持……加快发展，推动……。
    【展望】“山重水复疑无路，柳暗花明又一村。”路就在脚下，光明就在前方。我们希望越来越多优秀企业能够在我市共享发展机遇和优势，不断创造新的辉煌。我们将竭尽全力推动持续赋能企业发展，一如既往地为企业提供保姆式的服务和保障，支持企业在合作区实现高质量发展、再创新辉煌。
    【致谢】最后，预祝今天的活动圆满成功！谢谢大家。</article>
    【背景信息】{}
    模仿以上【示例文本】的文风，按照【称谓+欢迎+目的+站位引用+论点+发展历程+现状+做法1+做法2+做法3+展望+致谢】的顺序，写一篇关于【背景信息】的活动致辞，发言人是深圳市市长，重要嘉宾是王安石部长。请按照中国政府公文文风书写，字数在1000字左右。
    '''.format(info)
    return llm_6b(prompt4)

def llm_6b(prompt):
    response = openai.ChatCompletion.create(
        model="chatglm3-6b",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temerature=0.95
    )
    return response["choices"][0]["message"]["content"]

In [3]:
tools = [
    {
        "name": "google_search",
        "description": "在谷歌搜索引擎中进行查询",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {
                    "description": "需要搜索的问题",
                    'type': 'string',
                    "required": ['question']
                }
            }
        }
    },
    {
        "name": "extract_info",
        "description": "当用户输入活动的信息时，帮助用户提取信息",
        "parameters": {
            "type": "object",
            "properties": {
                "user_input": {
                    "description": "用户输入的活动信息",
                    'type': 'string',
                    "required": ['user_input']
                }
            }
        }
    },
    {
        "name": "opinion",
        "description": "以“补充活动信息”开头时，提供与活动相关的观点",
        "parameters": {
            "type": "object",
            "properties": {
                "info": {
                    "description": "所有活动信息",
                    'type': 'string',
                    "required": ['info']
                }
            }
        }
    }
]

In [4]:
import openai

def run_conv_glm(messages, functions_list=None, functions=None, model="chatglm3-6b", return_function_call=True):
    """
    能够自动执行外部函数调用的Chat对话模型
    :param messages: 必要参数，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为chatglm3-6b
    :return：Chat模型输出结果
    """
    def is_json_string(s):
        try:
            json_object = json.loads(s)
        except ValueError as e:
            return False
        return True

    user_messages = messages
    function_name = None

    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = openai.ChatCompletion.create(
            model=model,
            messages=user_messages,
            )
        response_message = response["choices"][0]["message"]
        final_response = response_message["content"]
        history = response["choices"][0]["history"]

    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建调用外部函数的system_message
        system_info = [
            {
                "role": "system",
                "content": "Answer the following questions as best as you can. You have access to the following tools:",
                "tools": functions
            }
        ]
        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}
        
        # 创建包含用户问题的message
        messages = system_info + user_messages

        # first response
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            return_function_call=return_function_call
        )
        response_message = response["choices"][0]["message"]
        history = response["choices"][0]["history"]

        # 判断返回结果是否为json格式对象，即判断是否需要调用外部函数来回答问题
        if is_json_string(response_message["content"]):
            # 需要调用外部函数
            function_call = json.loads(response.choices[0].message.content)
            # 获取函数名
            function_name = function_call["name"]
            # 获取函数对象
            fuction_to_call = available_functions[function_name]
            # 获取函数参数
            function_args = function_call['parameters']
            # 将函数参数输入到函数中，获取函数计算结果
            function_response = fuction_to_call(**function_args)
            
            # messages中拼接first response消息           
            messages.append(response_message) 
            # messages中拼接函数输出结果
            messages.append(
                {
                    "role": "observation",
                    "name": function_name,
                    "content": function_response,
                }
            ) 
            # 第二次调用模型
            second_response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            ) 
            # 获取最终结果
            # print(second_response)
            final_response = second_response["choices"][0]["message"]["content"]
            # history = second_response["choices"][0]["history"]
        else:
            final_response = response_message["content"]
            # history = response["choices"][0]["history"]

    # 新增一个字段来存储被调用的函数名
    called_function_name = None
    called_function_args = None
    if function_name:
        called_function_name = function_name
        called_function_args = function_args

    # 返回最终结果和被调用的函数名
    return final_response, called_function_name, called_function_args, history

In [5]:
def chat_with_glm(functions_list=None,
                  functions=None,
                  model="chatglm3-6b",
                  return_function_call=True):

    next_question_prompt = '''
    您好，我是您的AI写作助手，如果您需要写活动致辞，请将活动的相关信息告诉我，或者先对活动的内容进行搜索。
    如果提供信息，请按照“活动信息：……”进行输入；
    如果需要搜索，请按照“我想搜索：……”进行输入
    '''
    question = input(next_question_prompt)
    messages = [{"role": "user", "content": question}]
    conversation_log = []  # 初始化对话日志
    history = None
    search_count = 0 # 搜索次数
    
    while True: 
        # 方法一
        # 只保留最近的n轮对话
        n = 0
        n1 = n*2+1
        print(messages)
        if len(messages) > n1+1:
            messages = messages[-n1:]
        print(messages)
        
        # # 方法二
        # # 从后往前遍历，保留使history字符数刚好超过limit的那条消息
        # total_chars = 0
        # end_index = len(messages) - 1

        # # 从后往前累加字符数
        # limit = 200
        # for i in range(len(messages) - 1, -1, -1):
        #     total_chars += len(messages[i]["content"])
        #     if total_chars > limit:
        #         end_index = i
        #         break

        # # 保留从使history刚好超过limit的那条消息开始到列表末尾的所有消息
        # if total_chars > limit:
        #     messages = messages[end_index:]

        answer, called_function, function_args, history = run_conv_glm(messages=messages,
                                                functions_list=functions_list,  
                                                functions=functions,
                                                model=model,
                                                return_function_call=return_function_call)
        print(f"模型回答: {answer}")
        print(f"function: {called_function}")
        print(f"args: {function_args}")
        # print(messages)

        # 处理模型回答和历史记录
        history_log = []  # 初始化历史记录的列表
        if history:
            for item in history:
                role = item.get("role", "未知角色")
                content = item.get("content", "")
                # print(f"{role}: {content}")
                history_log.append({"role": role, "content": content})  # 将历史信息添加到列表
                # history_log = history_log[:-2]

        # 记录模型的回答和历史
        conversation_log.append({
            "model_ask": next_question_prompt,
            "user": question,
            "model_answer": answer,
            "function": called_function,
            "args": function_args,
            "history": history_log  # 将历史记录添加到对话日志
        })

        # 根据调用的函数定制下一步的提示
        if called_function == "google_search":
            if search_count < 1:
                next_question_prompt = "这是您的搜索结果，您还需要进行搜索吗？"
                question = input(next_question_prompt)
                search_count += 1
            else:
                next_question_prompt = "这是您的搜索结果，接下来请将活动信息告诉我"
                question = "请提取信息：\n" + input(next_question_prompt)

        elif called_function == "extract_info": 
            next_question_prompt = inquiry(answer)
            question = "补充活动信息：" + input(next_question_prompt)

        elif called_function == "opinion":
            next_question_prompt = "以上观点可供您参考"
            question = input(next_question_prompt)
            
        else:
            next_question_prompt = "您还有其他问题吗？(输入退出以结束对话): "
            question = input(next_question_prompt)

        if question == "退出":
            break
        messages.append({"role": "assistant", "content": answer})
        messages.append({"role": "user", "content": question})
    
    del messages
    return conversation_log  # 返回对话日志

In [67]:
chat_with_glm(functions_list=[google_search, extract_info, opinion],
              functions=tools)

[{'role': 'user', 'content': '我想搜索横琴中医药'}]
[{'role': 'user', 'content': '我想搜索横琴中医药'}]
模型回答: 
 根据您的搜索，我找到了以下几个与横琴中医药相关的信息：粤澳合作中医药科技产业园、横琴粤澳合作中医药科技产业园助力青年圆梦、【中新网】横琴粤澳深度合作区中医药产业加速发展、yyyymmd.cn、横琴新产业：中医药的品牌化之路、【粤开策略深度】横琴粤澳中医药产业图谱，十年一剑，跨越收展。请您参考这些信息了解更多关于横琴中医药的情况。
function: google_search
args: {'question': '横琴中医药'}
[{'role': 'user', 'content': '我想搜索横琴中医药'}, {'role': 'assistant', 'content': '\n 根据您的搜索，我找到了以下几个与横琴中医药相关的信息：粤澳合作中医药科技产业园、横琴粤澳合作中医药科技产业园助力青年圆梦、【中新网】横琴粤澳深度合作区中医药产业加速发展、yyyymmd.cn、横琴新产业：中医药的品牌化之路、【粤开策略深度】横琴粤澳中医药产业图谱，十年一剑，跨越收展。请您参考这些信息了解更多关于横琴中医药的情况。'}, {'role': 'user', 'content': '我想搜索横琴中医药近期的发展情况'}]
[{'role': 'user', 'content': '我想搜索横琴中医药'}, {'role': 'assistant', 'content': '\n 根据您的搜索，我找到了以下几个与横琴中医药相关的信息：粤澳合作中医药科技产业园、横琴粤澳合作中医药科技产业园助力青年圆梦、【中新网】横琴粤澳深度合作区中医药产业加速发展、yyyymmd.cn、横琴新产业：中医药的品牌化之路、【粤开策略深度】横琴粤澳中医药产业图谱，十年一剑，跨越收展。请您参考这些信息了解更多关于横琴中医药的情况。'}, {'role': 'user', 'content': '我想搜索横琴中医药近期的发展情况'}]
模型回答: 
 横琴中医药近期的发展情况主要表现在以下方面：1.橫琴粤澳中医药产业加速发展。2023年8月，大桥口岸药检中心通过验收，意味着横琴粤澳中医药产业�
function: g

[{'model_ask': '\n    您好，我是您的AI写作助手，如果您需要写活动致辞，请将活动的相关信息告诉我，或者先对活动的内容进行搜索。\n    如果提供信息，请按照“活动信息：……”进行输入；\n    如果需要搜索，请按照“我想搜索：……”进行输入\n    ',
  'user': '我想搜索横琴中医药',
  'model_answer': '\n 根据您的搜索，我找到了以下几个与横琴中医药相关的信息：粤澳合作中医药科技产业园、横琴粤澳合作中医药科技产业园助力青年圆梦、【中新网】横琴粤澳深度合作区中医药产业加速发展、yyyymmd.cn、横琴新产业：中医药的品牌化之路、【粤开策略深度】横琴粤澳中医药产业图谱，十年一剑，跨越收展。请您参考这些信息了解更多关于横琴中医药的情况。',
  'function': 'google_search',
  'args': {'question': '横琴中医药'},
  'history': [{'role': 'system',
    'content': 'Answer the following questions as best as you can. You have access to the following tools:'},
   {'role': 'user', 'content': '我想搜索横琴中医药'},
   {'role': 'assistant',
    'content': " ```python\ntool_call(question='横琴中医药')\n```"}]},
 {'model_ask': '这是您的搜索结果，您还需要进行搜索吗？',
  'user': '我想搜索横琴中医药近期的发展情况',
  'model_answer': '\n 横琴中医药近期的发展情况主要表现在以下方面：1.橫琴粤澳中医药产业加速发展。2023年8月，大桥口岸药检中心通过验收，意味着横琴粤澳中医药产业�',
  'function': 'google_search',
  'args': {'question': '横琴中医药近期的发展情况'},
  'history': [{'role': 'system',
    'content': 'Answer the follow

In [15]:
chat_with_glm(functions_list=[google_search, extract_info, opinion],
              functions=tools)

[{'role': 'user', 'content': '我想搜索新能源汽车'}]
[{'role': 'user', 'content': '我想搜索新能源汽车'}]
模型回答: 
 根据您的搜索，“新能源汽车”是一个比较广泛的概念，您可以通过以下链接获得更具体的回答：1. 新能源车 - 维基百科，自由的百科全书 <https://zh.wikipedia.org/zh-hans/%E6%96%B0%E8%83%BD%E6%BA%90%E8%BD%A6>2. 新能源汽车 - 百度百科 <https://baike.baidu.com/item/%E6%96%B0%E8%83%BD%E6%BA%90%E6%B1%BD%E8%BD%A6/2149544>3. 新能源车 - Autohome <https://ev.autohome.com.cn/>4. 中国新能源汽车亟待“标准”护航 - 新华网 <http://www.xinhuanet.com/auto/20240102/00fe9d827ab64e8585c3448f409db39a/c.html>5. 新能源汽车-比亚迪 <https://www.bydglobal.com/cn/ProductAndSolutions/car_mob.html>6. 新能源汽车概述 - Afcdud <https://www.afcdud.com/cn/transport-afcdud-3/400-2015-11-25-23-05-47.html>7. 2023最新新能源汽车图片及报价 - 汽车之家 <https://db.auto.sina.cn/estation/>希望这些链接能够满足您的需求。
function: google_search
args: {'question': '新能源汽车'}
[{'role': 'user', 'content': '我想搜索新能源汽车'}, {'role': 'assistant', 'content': '\n 根据您的搜索，“新能源汽车”是一个比较广泛的概念，您可以通过以下链接获得更具体的回答：1. 新能源车 - 维基百科，自由的百科全书 <https://zh.wikipedia.org/zh-hans/%E6%96%B0%E8%83%BD%E6%BA%90%E8%BD%A

[{'model_ask': '\n    您好，我是您的AI写作助手，如果您需要写活动致辞，请将活动的相关信息告诉我，或者先对活动的内容进行搜索。\n    如果提供信息，请按照“活动信息：……”进行输入；\n    如果需要搜索，请按照“我想搜索：……”进行输入\n    ',
  'user': '我想搜索新能源汽车',
  'model_answer': '\n 根据您的搜索，“新能源汽车”是一个比较广泛的概念，您可以通过以下链接获得更具体的回答：1. 新能源车 - 维基百科，自由的百科全书 <https://zh.wikipedia.org/zh-hans/%E6%96%B0%E8%83%BD%E6%BA%90%E8%BD%A6>2. 新能源汽车 - 百度百科 <https://baike.baidu.com/item/%E6%96%B0%E8%83%BD%E6%BA%90%E6%B1%BD%E8%BD%A6/2149544>3. 新能源车 - Autohome <https://ev.autohome.com.cn/>4. 中国新能源汽车亟待“标准”护航 - 新华网 <http://www.xinhuanet.com/auto/20240102/00fe9d827ab64e8585c3448f409db39a/c.html>5. 新能源汽车-比亚迪 <https://www.bydglobal.com/cn/ProductAndSolutions/car_mob.html>6. 新能源汽车概述 - Afcdud <https://www.afcdud.com/cn/transport-afcdud-3/400-2015-11-25-23-05-47.html>7. 2023最新新能源汽车图片及报价 - 汽车之家 <https://db.auto.sina.cn/estation/>希望这些链接能够满足您的需求。',
  'function': 'google_search',
  'args': {'question': '新能源汽车'},
  'history': [{'role': 'system',
    'content': 'Answer the following questions as best as you can. You have

In [8]:
chat_with_glm(functions_list=[google_search, extract_info, opinion],
              functions=tools)

[{'role': 'user', 'content': '我想搜索通用人工智能'}]
[{'role': 'user', 'content': '我想搜索通用人工智能'}]
模型回答: 
 根据您的搜索，我找到了以下信息：
function: google_search
args: {'question': '通用人工智能'}
[{'role': 'user', 'content': '我想搜索通用人工智能'}, {'role': 'assistant', 'content': '\n 根据您的搜索，我找到了以下信息：'}, {'role': 'user', 'content': '我想搜索通用人工智能近期的发展情况'}]
[{'role': 'user', 'content': '我想搜索通用人工智能近期的发展情况'}]
模型回答: 
 根据最近的新闻和研究报告，通用人工智能（AGI）正在朝着更加智能化和自主化的方向发展。例如，中国人工智能行业概览显示，我国人工智能市场规模不断扩大，应用范围逐渐深入；2023年人工智能现状报告显示，人工智能技术已经广泛应用于多个领域，但同时也面临着一些问题和挑战，例如数据隐私、伦理道德等。此外，有研究指出，通用人工智能的发展对算力提出了新的更高要求。因此，我们需要不断关注和研究通用人工智能的发展动态，以应对潜在的挑战和机遇。
function: google_search
args: {'question': '通用人工智能近期发展情况'}
[{'role': 'user', 'content': '我想搜索通用人工智能近期的发展情况'}, {'role': 'assistant', 'content': '\n 根据最近的新闻和研究报告，通用人工智能（AGI）正在朝着更加智能化和自主化的方向发展。例如，中国人工智能行业概览显示，我国人工智能市场规模不断扩大，应用范围逐渐深入；2023年人工智能现状报告显示，人工智能技术已经广泛应用于多个领域，但同时也面临着一些问题和挑战，例如数据隐私、伦理道德等。此外，有研究指出，通用人工智能的发展对算力提出了新的更高要求。因此，我们需要不断关注和研究通用人工智能的发展动态，以应对潜在的挑战和机遇。'}, {'role': 'user', 'content': '请提取信息：\n全球通用人工智

[{'model_ask': '\n    您好，我是您的AI写作助手，如果您需要写活动致辞，请将活动的相关信息告诉我，或者先对活动的内容进行搜索。\n    如果提供信息，请按照“活动信息：……”进行输入；\n    如果需要搜索，请按照“我想搜索：……”进行输入\n    ',
  'user': '我想搜索通用人工智能',
  'model_answer': '\n 根据您的搜索，我找到了以下信息：',
  'function': 'google_search',
  'args': {'question': '通用人工智能'},
  'history': [{'role': 'system',
    'content': 'Answer the following questions as best as you can. You have access to the following tools:'},
   {'role': 'user', 'content': '我想搜索通用人工智能'},
   {'role': 'assistant',
    'content': " ```python\ntool_call(question='通用人工智能')\n```"}]},
 {'model_ask': '这是您的搜索结果，您还需要进行搜索吗？',
  'user': '我想搜索通用人工智能近期的发展情况',
  'model_answer': '\n 根据最近的新闻和研究报告，通用人工智能（AGI）正在朝着更加智能化和自主化的方向发展。例如，中国人工智能行业概览显示，我国人工智能市场规模不断扩大，应用范围逐渐深入；2023年人工智能现状报告显示，人工智能技术已经广泛应用于多个领域，但同时也面临着一些问题和挑战，例如数据隐私、伦理道德等。此外，有研究指出，通用人工智能的发展对算力提出了新的更高要求。因此，我们需要不断关注和研究通用人工智能的发展动态，以应对潜在的挑战和机遇。',
  'function': 'google_search',
  'args': {'question': '通用人工智能近期发展情况'},
  'history': [{'role': 'system',
    'content': 'Answer the followin

In [8]:
chat_with_glm(functions_list=[google_search, extract_info, opinion],
              functions=tools)

[{'role': 'user', 'content': '我想搜索乡村振兴'}]
[{'role': 'user', 'content': '我想搜索乡村振兴'}]
模型回答: 
 针对您的查询“乡村振兴”，我为您找到了以下相关内容：中共中央国务院印发《乡村振兴战略规划（2018－2022年）》，国家乡村振兴局，中共中央国务院关于做好2023年全面推进乡村振兴重点工作等。您可以点击链接查看详细信息。
function: google_search
args: {'question': '乡村振兴'}
[{'role': 'user', 'content': '我想搜索乡村振兴'}, {'role': 'assistant', 'content': '\n 针对您的查询“乡村振兴”，我为您找到了以下相关内容：中共中央国务院印发《乡村振兴战略规划（2018－2022年）》，国家乡村振兴局，中共中央国务院关于做好2023年全面推进乡村振兴重点工作等。您可以点击链接查看详细信息。'}, {'role': 'user', 'content': '我想搜索乡村振兴目前实施的具体措施'}]
[{'role': 'user', 'content': '我想搜索乡村振兴目前实施的具体措施'}]
模型回答: 
 根据引擎返回的结果，目前实施的具体措施包括以下方面：
function: google_search
args: {'question': '乡村振兴目前实施的具体措施'}
[{'role': 'user', 'content': '我想搜索乡村振兴目前实施的具体措施'}, {'role': 'assistant', 'content': '\n 根据引擎返回的结果，目前实施的具体措施包括以下方面：'}, {'role': 'user', 'content': '请提取信息：\n全国乡村振兴战略实施研讨会，发言人是桃花源县管委会刘备主任，重要嘉宾是农业农村部关羽部长'}]
[{'role': 'user', 'content': '请提取信息：\n全国乡村振兴战略实施研讨会，发言人是桃花源县管委会刘备主任，重要嘉宾是农业农村部关羽部长'}]
模型回答: 
 非常感谢您的提问。根据您提供的信息，我为您提取了以下活动信息：全国乡村振兴战略实施研讨会，该活动由桃花财源管委刘备主任担

[{'model_ask': '\n    您好，我是您的AI写作助手，如果您需要写活动致辞，请将活动的相关信息告诉我，或者先对活动的内容进行搜索。\n    如果提供信息，请按照“活动信息：……”进行输入；\n    如果需要搜索，请按照“我想搜索：……”进行输入\n    ',
  'user': '我想搜索乡村振兴',
  'model_answer': '\n 针对您的查询“乡村振兴”，我为您找到了以下相关内容：中共中央国务院印发《乡村振兴战略规划（2018－2022年）》，国家乡村振兴局，中共中央国务院关于做好2023年全面推进乡村振兴重点工作等。您可以点击链接查看详细信息。',
  'function': 'google_search',
  'args': {'question': '乡村振兴'},
  'history': [{'role': 'system',
    'content': 'Answer the following questions as best as you can. You have access to the following tools:'},
   {'role': 'user', 'content': '我想搜索乡村振兴'},
   {'role': 'assistant',
    'content': " ```python\ntool_call(question='乡村振兴')\n```"}]},
 {'model_ask': '这是您的搜索结果，您还需要进行搜索吗？',
  'user': '我想搜索乡村振兴目前实施的具体措施',
  'model_answer': '\n 根据引擎返回的结果，目前实施的具体措施包括以下方面：',
  'function': 'google_search',
  'args': {'question': '乡村振兴目前实施的具体措施'},
  'history': [{'role': 'system',
    'content': 'Answer the following questions as best as you can. You have access to the following tools:'},
   {'role': 'user', 'conte

In [9]:
chat_with_glm(functions_list=[google_search, extract_info, opinion],
              functions=tools)

[{'role': 'user', 'content': '我想搜索绿色发展'}]
[{'role': 'user', 'content': '我想搜索绿色发展'}]
模型回答: 
 根据您的搜索结果，我为您找到了以下关于绿色发展的资料：新时代的中国绿色发展_白皮书、中国的绿色发展对世界意味着什么、百度百科关于绿色发展的词条以及多个新闻报道和政府网站的相关内容。您可以点击链接查看详细信息。
function: google_search
args: {'question': '绿色发展'}
[{'role': 'user', 'content': '我想搜索绿色发展'}, {'role': 'assistant', 'content': '\n 根据您的搜索结果，我为您找到了以下关于绿色发展的资料：新时代的中国绿色发展_白皮书、中国的绿色发展对世界意味着什么、百度百科关于绿色发展的词条以及多个新闻报道和政府网站的相关内容。您可以点击链接查看详细信息。'}, {'role': 'user', 'content': '我想搜索绿色发展战略的重要举措'}]
[{'role': 'user', 'content': '我想搜索绿色发展战略的重要举措'}]
模型回答: 
 根据搜索结果，绿色发展战略的重要举措包括制定促进经济社会发展全面绿色转型的政策举措、加快构建绿色发展的支撑体系、加快推广绿色技术和绿色产品等。此外，还有许多具体的活动和计划，例如第三届“一带一路”国际合作高峰论坛绿色发展高级别会议、中国绿色发展为全球生态治理作出重要贡献等等。可以查看上述链接获取更多详细信息。
function: google_search
args: {'question': '绿色发展战略的重要举措'}
[{'role': 'user', 'content': '我想搜索绿色发展战略的重要举措'}, {'role': 'assistant', 'content': '\n 根据搜索结果，绿色发展战略的重要举措包括制定促进经济社会发展全面绿色转型的政策举措、加快构建绿色发展的支撑体系、加快推广绿色技术和绿色产品等。此外，还有许多具体的活动和计划，例如第三届“一带一路”国际合作高峰论坛绿色发展高级别会议、中国绿色发展为全球生态治理作出重要贡献等等。可以查看上述链接获取更多详细信息。'}, {

[{'model_ask': '\n    您好，我是您的AI写作助手，如果您需要写活动致辞，请将活动的相关信息告诉我，或者先对活动的内容进行搜索。\n    如果提供信息，请按照“活动信息：……”进行输入；\n    如果需要搜索，请按照“我想搜索：……”进行输入\n    ',
  'user': '我想搜索绿色发展',
  'model_answer': '\n 根据您的搜索结果，我为您找到了以下关于绿色发展的资料：新时代的中国绿色发展_白皮书、中国的绿色发展对世界意味着什么、百度百科关于绿色发展的词条以及多个新闻报道和政府网站的相关内容。您可以点击链接查看详细信息。',
  'function': 'google_search',
  'args': {'question': '绿色发展'},
  'history': [{'role': 'system',
    'content': 'Answer the following questions as best as you can. You have access to the following tools:'},
   {'role': 'user', 'content': '我想搜索绿色发展'},
   {'role': 'assistant',
    'content': " ```python\ntool_call(question='绿色发展')\n```"}]},
 {'model_ask': '这是您的搜索结果，您还需要进行搜索吗？',
  'user': '我想搜索绿色发展战略的重要举措',
  'model_answer': '\n 根据搜索结果，绿色发展战略的重要举措包括制定促进经济社会发展全面绿色转型的政策举措、加快构建绿色发展的支撑体系、加快推广绿色技术和绿色产品等。此外，还有许多具体的活动和计划，例如第三届“一带一路”国际合作高峰论坛绿色发展高级别会议、中国绿色发展为全球生态治理作出重要贡献等等。可以查看上述链接获取更多详细信息。',
  'function': 'google_search',
  'args': {'question': '绿色发展战略的重要举措'},
  'history': [{'role': 'system',
    'content': 'A

In [10]:
chat_with_glm(functions_list=[google_search, extract_info, opinion],
              functions=tools)

[{'role': 'user', 'content': '我想搜索防灾减灾的重要性'}]
[{'role': 'user', 'content': '我想搜索防灾减灾的重要性'}]
模型回答: 
 非常感谢您的提问。根据我的了解，防灾减灾具有重要意义。通过防灾减灾，可以减少自然灾害对人类生命和财产的伤害和损失，促进社会经济的可持续发展。在我国，防灾减灾工作已经得到越来越多的重视，各级政府和相关部门都在积极推动防灾减灾工作的开展。希望通过大家的共同努力，我们能够更好地应对自然灾害，减少灾害对我们生活的影响。
function: google_search
args: {'question': '防灾减灾的重要性'}
[{'role': 'user', 'content': '我想搜索防灾减灾的重要性'}, {'role': 'assistant', 'content': '\n 非常感谢您的提问。根据我的了解，防灾减灾具有重要意义。通过防灾减灾，可以减少自然灾害对人类生命和财产的伤害和损失，促进社会经济的可持续发展。在我国，防灾减灾工作已经得到越来越多的重视，各级政府和相关部门都在积极推动防灾减灾工作的开展。希望通过大家的共同努力，我们能够更好地应对自然灾害，减少灾害对我们生活的影响。'}, {'role': 'user', 'content': '我想搜索防灾减灾的必要性'}]
[{'role': 'user', 'content': '我想搜索防灾减灾的必要性'}]
模型回答: 
 根据我的搜索结果，我将为您总结一下灾害预防重要性的几个方面。首先，灾害预防可以减少财产损失和人员伤亡。其次，灾害预防可以帮助加强社区、村庄和城市的抗灾能力，提高基础设施的抗震设防标准，降低防灾减灾成本。另外，灾害预防可以通过普及和安全意识的提高，减少错误的应对措施和灾难的发生。最后，政府应该建立完善的防灾减灾体制机制，加强灾害监测预警和应急响应能力，确保人民群众的生命财产安全。
function: google_search
args: {'question': '防灾减灾的必要性'}
[{'role': 'user', 'content': '我想搜索防灾减灾的必要性'}, {'role': 'assistant', 'content': '\n 根据我的搜索结果，我将为您总结一下

[{'model_ask': '\n    您好，我是您的AI写作助手，如果您需要写活动致辞，请将活动的相关信息告诉我，或者先对活动的内容进行搜索。\n    如果提供信息，请按照“活动信息：……”进行输入；\n    如果需要搜索，请按照“我想搜索：……”进行输入\n    ',
  'user': '我想搜索防灾减灾的重要性',
  'model_answer': '\n 非常感谢您的提问。根据我的了解，防灾减灾具有重要意义。通过防灾减灾，可以减少自然灾害对人类生命和财产的伤害和损失，促进社会经济的可持续发展。在我国，防灾减灾工作已经得到越来越多的重视，各级政府和相关部门都在积极推动防灾减灾工作的开展。希望通过大家的共同努力，我们能够更好地应对自然灾害，减少灾害对我们生活的影响。',
  'function': 'google_search',
  'args': {'question': '防灾减灾的重要性'},
  'history': [{'role': 'system',
    'content': 'Answer the following questions as best as you can. You have access to the following tools:'},
   {'role': 'user', 'content': '我想搜索防灾减灾的重要性'},
   {'role': 'assistant',
    'content': " ```python\ntool_call(question='防灾减灾的重要性')\n```"}]},
 {'model_ask': '这是您的搜索结果，您还需要进行搜索吗？',
  'user': '我想搜索防灾减灾的必要性',
  'model_answer': '\n 根据我的搜索结果，我将为您总结一下灾害预防重要性的几个方面。首先，灾害预防可以减少财产损失和人员伤亡。其次，灾害预防可以帮助加强社区、村庄和城市的抗灾能力，提高基础设施的抗震设防标准，降低防灾减灾成本。另外，灾害预防可以通过普及和安全意识的提高，减少错误的应对措施和灾难的发生。最后，政府应该建立完善的防灾减灾体制机制，加强灾害监测预警和应急响应能力，确保人民群众的生命财产安全。',
  'function': 'googl